# Introduction 
This is my solution that I am glad to share.This solution got me to the top 3 % in the private leaderboard..so I hope you benefit from it.. My solution is an ensemble of 3 Neural Network Models 2 CNNs with nearly same architecture and different hyperparameters and 1 RNN..then I blended the 3 predictions using a simple linear regressor

## Import dependencies

In [ ]:
%%time
import pandas as pd
import numpy as np
from nltk.corpus import stopwords
from keras.preprocessing.text import Tokenizer
from keras.layers import GRU,Conv1D,MaxPooling1D,Input,Dense,Dropout
from sklearn.metrics import mean_squared_error
import datetime
time_start=datetime.datetime.now()
from tensorflow import set_random_seed
import tensorflow as tf
session_conf = tf.ConfigProto(intra_op_parallelism_threads=10, inter_op_parallelism_threads=1)
from keras import backend
backend.set_session(tf.Session(graph=tf.get_default_graph(), config=session_conf))
from collections import Counter
import os
os.environ['PYTHONHASHSEED'] = '123'
np.random.seed(123)
import random
np.random.seed(123)
random.seed(123)
set_random_seed(123)

# Credit and Big Thanks
This kernel https://www.kaggle.com/valkling/mercari-rnn-2ridge-models-with-notes-0-42755 has helped me a lot specially about the feature engineering,NN architecture and ensembling..Long story short..it was such an inspiration so big thanks for such kernel

## Reading train and test data

In [ ]:
%%time
np.random.seed(123)
train = pd.read_csv('../input/train.tsv',sep='\t')
test = pd.read_csv('../input/test.tsv',sep='\t')
train=train[(train.price>=3)]
#train=train[train.price<2000]
full_df=pd.concat([train,test],ignore_index=True,axis=0)

## Preprocess Data

In [ ]:
%%time
all_brands = set(full_df['brand_name'].values)
full_df.brand_name.fillna('missing',inplace=True)
# get to finding!
premissing = len(full_df.loc[full_df.brand_name == 'missing'])
def brandfinder(line):
    brand = line[0]
    name = line[1]
    namesplit = name.split(' ')
    if brand == 'missing':
        for x in namesplit:
            if x in all_brands:
                return name
    if name in all_brands:
        return name
    return brand
full_df['brand_name'] = full_df[['brand_name','name']].apply(brandfinder, axis = 1)
found = premissing-len(full_df.loc[full_df['brand_name'] == 'missing'])
print(found)

In [ ]:
%%time
def preprocess_data(df):
    df.item_description.replace('No description yet',value='missing',inplace=True)
    df.item_description.fillna('missing',inplace=True)
    #df.brand_name.fillna('missing',inplace=True)
    df.category_name.fillna('missing/missing/missing',inplace=True)
    df['main_category']=df.category_name.apply(lambda x:x.split('/')[0])
    df['sub_category1']=df.category_name.apply(lambda x:x.split('/')[1])
    df['sub_category2']=df.category_name.apply(lambda x:x.split('/')[2])
preprocess_data(full_df)

## Tokenizer

## Notes
* I tried to preprocess text and remove stopwords using regex since removing them through NLTK was too time consuming and time is an important factor in such competition.
* Also stemming and lemmatizing text didn't work because of the same reason and I am not sure if it would help the models. 
* Another surprising thing to me that the models performance were better when initializing embeddings from scratch other tha using pretrained vectors as Word2Vec/Glove/FastText 
* I concatenated the words from train and test when fitting keras tokenizer and chose the maximum number of words removing only the words that occured once
* I used the description length and name length as features

In [ ]:
%%time
print('preprocessing Text')
#convert to lowercase
#full_df.item_description=full_df.item_description.apply(lambda x:x.lower())
#full_df.name=full_df.name.apply(lambda x:x.lower())
#full_df.item_description.replace('\W+',' ',regex=True,inplace=True)
#full_df.item_description.replace('\s+',' ',inplace=True,regex=True)
#full_df.name.replace('\W+',' ',regex=True,inplace=True)
#full_df.name.replace('\s+',' ',inplace=True,regex=True)
#full_df.item_description.replace(r'\bt\sshirt\b','t-shirt',inplace=True,regex=True)
#full_df.name.replace(r'\bt\sshirt\b',' t-shirt',inplace=True,regex=True)
#full_df.item_description.replace('\sgb','gb',inplace=True,regex=True)
#full_df.name.replace('\sgb','gb',inplace=True,regex=True)
#full_df.item_description.replace('[rm]',' ',inplace=True)
#full_df.item_description.replace('rm',' ',inplace=True)
#full_df.item_description.replace('.*\d+.*',' ',regex=True,inplace=True)
#full_df.name.replace('.*\d+.*',' ',regex=True,inplace=True)
#full_df.item_description.replace('\s+',' ',inplace=True,regex=True)

#stopwords=stopwords.words('english')
#pat = r'\b(?:{})\b'.format('|'.join(stopwords))
#full_df.item_description.replace(pat,' ',inplace=True,regex=True)
#full_df.name.replace(pat,' ',inplace=True,regex=True)
raw_text=np.hstack([full_df.item_description.str.lower(),full_df.name.str.lower()])
print('Fitting Tokenzier')
tokenizer=Tokenizer()
tokenizer.fit_on_texts(raw_text)
word_index=tokenizer.word_index
word_counts=tokenizer.word_counts
c=Counter(sorted(word_counts.values()))
tokenizer.num_words=len(word_index)-c[1]
NUM_WORDS=tokenizer.num_words
#tokenizer_cat=Tokenizer()
#tokenizer_cat.fit_on_texts(category)
print('transforming to sequences')
n_train=train.shape[0]
full_df['seq_name']=tokenizer.texts_to_sequences(full_df.name.str.lower())
full_df['seq_desc']=tokenizer.texts_to_sequences(full_df.item_description.str.lower())
#full_df['category_seq']=tokenizer_cat.texts_to_sequences(full_df.category_nameen
full_df['name_len']=full_df.seq_name.apply(lambda x: len(x))
full_df['desc_len']=full_df.seq_desc.apply(lambda x:len(x))

del tokenizer
del word_index
del c
del word_counts

## Label Encoding

In [ ]:
%%time
from sklearn.preprocessing import LabelEncoder
print("Handling categorical variables")
le=LabelEncoder()
full_df['category_name']=le.fit_transform(full_df.category_name)
full_df['brand_name'] = le.fit_transform(full_df.brand_name)
full_df['main_category']=le.fit_transform(full_df.main_category)
full_df['sub_category1']=le.fit_transform(full_df.sub_category1)
full_df['sub_category2']=le.fit_transform(full_df.sub_category2)
del le

## train test split

In [ ]:
%%time
from sklearn.model_selection import train_test_split
train=full_df[:n_train]
test=full_df[n_train:]
dtrain,dvalid=train_test_split(train,random_state=123,test_size=0.01)


## Embeddings MaxValues

In [ ]:
%%time
MAX_NAME_SEQ = 10
MAX_ITEM_DESC_SEQ = 75
#MAX_TEXT = np.max([np.max(train.seq_name.max())
#                   , np.max(test.seq_name.max())
#                  , np.max(train.seq_desc.max())
#                  , np.max(test.seq_desc.max())])+2
#MAX_TEXT=NUM_WORDS+1
if NUM_WORDS is None:
    MAX_TEXT=len(word_index)+1
else:
    MAX_TEXT=NUM_WORDS+1
MAX_CATEGORY_MAIN = np.max([full_df.main_category.max()])+1
MAX_CATEGORY_SUB1 = np.max([full_df.sub_category1.max()])+1
MAX_CATEGORY_SUB2 = np.max([full_df.sub_category2.max()])+1
MAX_BRAND = np.max([full_df.brand_name.max()])+1
MAX_CONDITION = np.max([full_df.item_condition_id.max()])+1
MAX_DESC_LEN = np.max([full_df.desc_len.max()])+1
MAX_NAME_LEN = np.max([full_df.name_len.max()])+1

## Keras Data Notes

* As you will notice there are two item condition variables called 'item_condition' and 'item_condition2'.One of them was one hot encoded as it helped the CNN models

## Get Keras data

In [ ]:
%%time
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical

def get_keras_data(dataset):
    X = {
        'name': pad_sequences(dataset.seq_name, maxlen=MAX_NAME_SEQ),
        'item_desc': pad_sequences(dataset.seq_desc, maxlen=MAX_ITEM_DESC_SEQ),
        'brand_name': np.array(dataset[["brand_name"]]),
        'item_condition': to_categorical(dataset[["item_condition_id"]]),
        'num_vars': np.array(dataset[["shipping"]]),
        'desc_len': np.array(dataset[["desc_len"]]),
        'name_len': np.array(dataset[["name_len"]]),
        'main_category':np.array(dataset[["main_category"]]),
        'sub_category1': np.array(dataset[["sub_category1"]]),
        'sub_category2':np.array(dataset[["sub_category2"]]),
        'category_name':np.array(dataset[["category_name"]]),
        'item_condition2':np.array(dataset[["item_condition_id"]]),
    }
    return X

X_train = get_keras_data(dtrain)
y_train=np.log1p(dtrain.price)
X_valid = get_keras_data(dvalid)
y_valid = np.log1p(dvalid.price)
X_test = get_keras_data(test)
test_id=test['test_id'].astype('int32')
del dtrain
del dvalid
del test
del train
del full_df

## NN Models
Surprisingly the 2 CNN models outperformed the RNN model in this specific dataset.Another important factor that helped me improve the score was the hyperparameter tuning specially the kernel initialization for embeddings and Fully connected layers.Batch Normalization for description length and name length improved the score for CNN.


**using dropout didn't improve the score so I set it to zero

In [ ]:
%%time
from keras.layers import Embedding,GlobalMaxPooling1D,concatenate,Flatten,SpatialDropout1D,Bidirectional,MaxPooling1D,LSTM,BatchNormalization,GlobalAveragePooling1D
from keras.utils import to_categorical
from keras.models import Model
from keras.optimizers import Adam,SGD,Adamax
from keras import regularizers
from keras.initializers import Constant
def get_cnn_model(lr=0.001,decay=0):
    dr_r=0
    #Inputs
    name=Input(shape=[X_train['name'].shape[1]],name='name')
    category_name=Input(shape=[1],name='category_name')
    desc=Input(shape=[X_train['item_desc'].shape[1]],name='item_desc')
    brand=Input(shape=[1],name='brand_name')
    item_condition=Input(shape=[X_train['item_condition'].shape[1]],name='item_condition')
    shipping=Input(shape=[X_train['num_vars'].shape[1]],name='num_vars')
    desc_len=Input(shape=[1],name='desc_len')
    name_len=Input(shape=[1],name='name_len')
    main_category=Input(shape=[1],name='main_category')
    sub_category1=Input(shape=[1],name='sub_category1')
    sub_category2=Input(shape=[1],name='sub_category2')
    #Embeddings
    emb_name=Embedding(MAX_TEXT,50,embeddings_initializer='glorot_uniform')(name)
    emb_desc=Embedding(MAX_TEXT,50,embeddings_initializer='glorot_uniform')(desc)
    emb_brand=Embedding(MAX_BRAND,10,embeddings_initializer='glorot_uniform')(brand)
    emb_desc_len=Embedding(MAX_DESC_LEN,5,embeddings_initializer='glorot_uniform')(desc_len)
    emb_name_len=Embedding(MAX_NAME_LEN,5,embeddings_initializer='glorot_uniform')(name_len)
    emb_main_category=Embedding(MAX_CATEGORY_MAIN,10,embeddings_initializer='glorot_uniform')(main_category)
    emb_sub_category1=Embedding(MAX_CATEGORY_SUB1,10,embeddings_initializer='glorot_uniform')(sub_category1)
    emb_sub_category2=Embedding(MAX_CATEGORY_SUB2,10,embeddings_initializer='glorot_uniform')(sub_category2)
    emb_item_condition =Embedding(MAX_CONDITION, 5,embeddings_initializer='glorot_uniform')(item_condition)
    #emb_category_name=BatchNormalization()(category_name)#Embedding(MAX_CATEGORY_TEXT,20)(category_name)
    #CNN
    conv_name=Conv1D(filters=32,kernel_size=3,activation=None,kernel_initializer='glorot_uniform')(emb_name)
    #conv_name=Conv1D(filters=16,kernel_size=3,activation='relu',kernel_initializer='glorot_uniform')(conv_name)
    
    max_pooling_name=GlobalMaxPooling1D()(conv_name)
    conv_desc=Conv1D(filters=64,kernel_size=3,activation=None,kernel_initializer='glorot_uniform')(emb_desc)
    #conv_desc=Conv1D(filters=32,kernel_size=3,activation='relu',kernel_initializer='glorot_uniform')(conv_desc)
    max_pooling_desc=GlobalMaxPooling1D()(conv_desc)
    #conv_category_name=Conv1D(filters=8,kernel_size=3,activation='relu')(emb_category_name)
    #max_pooling_category_name=GlobalMaxPooling1D()(conv_category_name)
    #conv_sub_category1=Conv1D(filters=8,kernel_size=3,activation='relu')(emb_sub_category1)
    #max_pooling_sub_category1=GlobalMaxPooling1D()(conv_sub_category1)
    #conv_sub_category2=Conv1D(filters=8,kernel_size=3,activation='relu')(emb_sub_category2)
    #max_pooling_sub_category2=GlobalMaxPooling1D()(conv_sub_category2)
    
    #Fully Connected Neural Netwo)rks
    main_l=concatenate([
        max_pooling_name,
        max_pooling_desc,
        Flatten()(emb_main_category),
        Flatten()(emb_sub_category1),
        Flatten()(emb_sub_category2),
        Flatten()(emb_name_len),
        Flatten()(emb_desc_len),
        Flatten()(emb_brand),
        item_condition,
        shipping,
    #    b_name_len2,
    #    b_desc_len2
    ])
    #main_l=BatchNormalization()(main_l)
    main_l=Dropout(dr_r)(Dense(256,activation='relu',kernel_initializer='he_uniform')(main_l))
    #main_l=BatchNormalization()(main_l)
    main_l=Dropout(dr_r)(Dense(128,activation='relu',kernel_initializer='he_uniform')(main_l))
    #main_l=BatchNormalization()(main_l)
    #main_l=Dropout(dr_r)(Dense(64,activation='relu')(main_l))

    output=Dense(1,kernel_initializer='he_uniform')(main_l)
    
    cnn_model=Model(inputs=[name,desc,brand,item_condition,shipping,desc_len,name_len,category_name,main_category,sub_category1,sub_category2],outputs=output)
    optimizer=Adam(lr=lr,decay=decay)
    cnn_model.compile(loss="mse", optimizer=optimizer)
    
    return cnn_model
def new_rnn_model(lr=0.001, decay=0.0):
    dr_r=0
    name = Input(shape=[X_train["name"].shape[1]], name="name")
    item_desc = Input(shape=[X_train["item_desc"].shape[1]], name="item_desc")
    brand_name = Input(shape=[1], name="brand_name")
    item_condition = Input(shape=[1], name="item_condition2")
    num_vars = Input(shape=[X_train["num_vars"].shape[1]], name="num_vars")
    desc_len = Input(shape=[1], name="desc_len")
    name_len = Input(shape=[1], name="name_len")
    subcat_0 = Input(shape=[1], name="main_category")
    subcat_1 = Input(shape=[1], name="sub_category1")
    subcat_2 = Input(shape=[1], name="sub_category2")
    #new inputs
    brand_enc=Input(shape=[1],name='brand_name_enc')
    item_condition_enc=Input(shape=[1],name='item_condition_enc')
    shipping_enc=Input(shape=[1],name='shipping_enc')
    main_category_enc=Input(shape=[1],name='main_category_enc')
    sub_category1_enc=Input(shape=[1],name='sub_category1_enc')
    sub_category2_enc=Input(shape=[1],name='sub_category2_enc')
    # Embeddings layers (adjust outputs to help model)
    emb_name = Embedding(MAX_TEXT, 50,name='emb_name',embeddings_initializer='glorot_uniform')(name)
    emb_item_desc = Embedding(MAX_TEXT, 50,name='emb_item_desc',embeddings_initializer='glorot_uniform')(item_desc)
    emb_brand_name = Embedding(MAX_BRAND, 10,name='emb_brand_name',embeddings_initializer='glorot_uniform')(brand_name)
    emb_item_condition = Embedding(MAX_CONDITION,5,name='emb_item_condition',embeddings_initializer='glorot_uniform')(item_condition)
    emb_desc_len = Embedding(MAX_DESC_LEN, 5,name='emb_desc_len',embeddings_initializer='glorot_uniform')(desc_len)
    emb_name_len = Embedding(MAX_NAME_LEN, 5,name='emb_name_len',embeddings_initializer='glorot_uniform')(name_len)
    emb_subcat_0 = Embedding(MAX_CATEGORY_MAIN, 10,name='emb_sub_cat_0',embeddings_initializer='glorot_uniform')(subcat_0)
    emb_subcat_1 = Embedding(MAX_CATEGORY_SUB1, 10,name='emb_sub_cat_1',embeddings_initializer='glorot_uniform')(subcat_1)
    emb_subcat_2 = Embedding(MAX_CATEGORY_SUB2, 10,name='emb_sub_cat_2',embeddings_initializer='glorot_uniform')(subcat_2)
    

    # rnn layers (GRUs are faster than LSTMs and speed is important here)
    #conv_name=Conv1D(filters=32,kernel_size=3,activation=None,kernel_initializer='glorot_uniform')(emb_name)
    #max_pooling_name=MaxPooling1D(pool_size=3)(conv_name)
    #conv_desc=Conv1D(filters=32,kernel_size=3,activation=None,kernel_initializer='glorot_uniform')(emb_item_desc)
    #max_pooling_desc=MaxPooling1D(pool_size=3)(conv_desc)
    rnn_layer1 = GRU(16,kernel_initializer='glorot_uniform') (emb_item_desc)
    #rnn_layer1 = GRU(16,kernel_initializer='glorot_uniform')(rnn_layer1)
    rnn_layer2 = GRU(8,kernel_initializer='glorot_uniform') (emb_name)
    #rnn_layer2 = GRU(8,kernel_initializer='glorot_uniform') (rnn_layer2)
    # main layers
    main_l = concatenate([
        Flatten() (emb_brand_name)
        , Flatten()(emb_item_condition)
        , Flatten()(emb_desc_len)
        , Flatten()(emb_name_len)
        , Flatten() (emb_subcat_0)
        , Flatten() (emb_subcat_1)
        , Flatten() (emb_subcat_2)
        , rnn_layer1
        , rnn_layer2
        , num_vars,
    ])
    
    # (incressing the nodes or adding layers does not effect the time quite as much as the rnn layers)
    main_l = Dropout(dr_r)(Dense(512,kernel_initializer='he_uniform',activation='relu') (main_l))
    main_l = Dropout(dr_r)(Dense(256,kernel_initializer='he_uniform',activation='relu') (main_l))
    main_l = Dropout(dr_r)(Dense(128,kernel_initializer='he_uniform',activation='relu') (main_l))
    main_l = Dropout(dr_r)(Dense(64,kernel_initializer='he_uniform',activation='relu') (main_l))
    #main_l = Dropout(dr_r)(Dense(32,kernel_initializer='he_uniform',activation='relu') (main_l))
    # the output layer.
    output = Dense(1,activation="linear") (main_l)
    
    model = Model([name, item_desc, brand_name , item_condition, 
                   num_vars, desc_len, name_len, subcat_0, subcat_1, 
                   subcat_2], output)

    optimizer = Adam(lr=lr, decay=decay)
    
    # (mean squared error loss function works as well as custom functions)  
    model.compile(loss = 'mse', optimizer = optimizer)

    return model
def get_cnn_model2(lr=0.001,decay=0):
    dr_r=0
    #Inputs
    name=Input(shape=[X_train['name'].shape[1]],name='name')
    category_name=Input(shape=[1],name='category_name')
    desc=Input(shape=[X_train['item_desc'].shape[1]],name='item_desc')
    brand=Input(shape=[1],name='brand_name')
    item_condition=Input(shape=[X_train['item_condition'].shape[1]],name='item_condition')
    shipping=Input(shape=[X_train['num_vars'].shape[1]],name='num_vars')
    desc_len=Input(shape=[1],name='desc_len')
    name_len=Input(shape=[1],name='name_len')
    main_category=Input(shape=[1],name='main_category')
    sub_category1=Input(shape=[1],name='sub_category1')
    sub_category2=Input(shape=[1],name='sub_category2')
    #Embeddings
    emb_name=Embedding(MAX_TEXT,50,embeddings_initializer='glorot_uniform')(name)
    emb_desc=Embedding(MAX_TEXT,50,embeddings_initializer='glorot_uniform')(desc)
    emb_brand=Embedding(MAX_BRAND,10,embeddings_initializer='glorot_uniform')(brand)
    emb_desc_len=BatchNormalization()(desc_len)
    emb_name_len=BatchNormalization()(name_len)
    emb_main_category=Embedding(MAX_CATEGORY_MAIN,10,embeddings_initializer='glorot_uniform')(main_category)
    emb_sub_category1=Embedding(MAX_CATEGORY_SUB1,10,embeddings_initializer='glorot_uniform')(sub_category1)
    emb_sub_category2=Embedding(MAX_CATEGORY_SUB2,10,embeddings_initializer='glorot_uniform')(sub_category2)
    emb_item_condition = Embedding(MAX_CONDITION, 5,embeddings_initializer='glorot_uniform')(item_condition)
    #emb_category_name=Embedding(MAX_CATEGORY_TEXT,20)(category_name)
    #CNN
    conv_name=Conv1D(filters=32,kernel_size=3,activation=None,kernel_initializer='glorot_uniform')(emb_name)
    #conv_name=SpatialDropout1D(0.5)(conv_name)
    #conv_name=SpatialDropout1D(dr_r)(conv_name)
    max_pooling_name=GlobalMaxPooling1D()(conv_name)
    conv_desc=Conv1D(filters=64,kernel_size=3,activation=None,kernel_initializer='glorot_uniform')(emb_desc)
    max_pooling_desc=GlobalMaxPooling1D()(conv_desc)
    #conv_category_name=Conv1D(filters=8,kernel_size=3,activation='relu')(emb_category_name)
    #max_pooling_category_name=GlobalMaxPooling1D()(conv_category_name)
    #conv_sub_category1=Conv1D(filters=8,kernel_size=3,activation='relu')(emb_sub_category1)
    #max_pooling_sub_category1=GlobalMaxPooling1D()(conv_sub_category1)
    #conv_sub_category2=Conv1D(filters=8,kernel_size=3,activation='relu')(emb_sub_category2)
    #max_pooling_sub_category2=GlobalMaxPooling1D()(conv_sub_category2)
    #Fully Connected Neural Networks
    main_l=concatenate([
        max_pooling_name,
        max_pooling_desc,
       # Flatten()(emb_category_name),
        Flatten()(emb_main_category),
        Flatten()(emb_sub_category1),
        Flatten()(emb_sub_category2),
        emb_name_len,
        emb_desc_len,
        Flatten()(emb_brand),
        item_condition,
        shipping
    ])
    main_l=Dropout(dr_r)(Dense(256,activation='relu',kernel_initializer='he_uniform')(main_l))
    main_l=Dropout(dr_r)(Dense(128,activation='relu',kernel_initializer='he_uniform')(main_l))
    #main_l=Dropout(dr_r)(Dense(64,activation='relu')(main_l))

    output=Dense(1,kernel_initializer='he_normal')(main_l)
    
    cnn_model=Model(inputs=[name,desc,brand,item_condition,shipping,desc_len,name_len,main_category,sub_category1,sub_category2,category_name],outputs=output)
    optimizer=Adam(lr=lr,decay=decay)
    cnn_model.compile(loss="mse", optimizer=optimizer)
    
    return cnn_model

    
    

In [ ]:
%%time
BATCH_SIZE = 512*3
epochs = 2
np.random.seed(123)
exp_decay = lambda init, fin, steps: (init/fin)**(1/(steps-1)) - 1
steps = int(len(X_train['name']) / BATCH_SIZE) * epochs
lr_init, lr_fin = 0.005, 0.0001
lr_decay = exp_decay(lr_init, lr_fin, steps)



model= get_cnn_model(lr_init,lr_decay)
#model.summary()
model.fit(X_train,y_train, epochs=epochs, batch_size=BATCH_SIZE
          , validation_data=(X_valid,y_valid)
          , verbose=1)

In [ ]:
%%time
np.random.seed(123)
BATCH_SIZE = 512*3
epochs = 2
lr_init, lr_fin = 0.004, 0.001
steps = int(len(X_train['name']) / BATCH_SIZE) * epochs
exp_decay = lambda init, fin, steps: (init/fin)**(1/(steps-1)) - 1
lr_decay = exp_decay(lr_init, lr_fin, steps)
model2= new_rnn_model(lr_init,lr_decay)
model2.fit(X_train,y_train, epochs=epochs, batch_size=BATCH_SIZE
          , validation_data=(X_valid,y_valid)
          , verbose=1)

In [ ]:
%%time
np.random.seed(123)
BATCH_SIZE = 512*3
epochs = 2
lr_init, lr_fin = 0.005, 0.001
steps = int(len(X_train['name']) / BATCH_SIZE) * epochs
exp_decay = lambda init, fin, steps: (init/fin)**(1/(steps-1)) - 1
lr_decay = exp_decay(lr_init, lr_fin, steps)
model3= get_cnn_model2(lr_init,lr_decay)
model3.fit(X_train,y_train, epochs=epochs, batch_size=BATCH_SIZE
          , validation_data=(X_valid,y_valid)
          , verbose=1)

#Ensembling
Here where we combined the 3 models and used Linear Regression for the best score..Ridge and RidgeCV were tried but didn't differ in the score so I chose the simple Linear Regression.. 

In [ ]:
%%time
from sklearn.linear_model import LinearRegression,Ridge,RidgeCV
cnn_predict=model.predict(X_valid,batch_size=BATCH_SIZE)
rnn_predict=model2.predict(X_valid,batch_size=BATCH_SIZE)
cnn_predict2=model3.predict(X_valid,batch_size=BATCH_SIZE)
X_meta=np.hstack([cnn_predict,rnn_predict,cnn_predict2])
lm=LinearRegression()
lm.fit(X_meta,y_valid)
new_pred=lm.predict(X_meta)
rmsle=np.sqrt(mean_squared_error(y_valid,new_pred))
print ('loss:'+str(rmsle))

In [ ]:
#%%time
#np.random.seed(123)
#cnn_predict=model.predict(X_valid,batch_size=BATCH_SIZE)
#rnn_predict=model3.predict(X_valid,batch_size=BATCH_SIZE)
#min_loss=2**23
#best_alpha=0
#for i in range(100):
#    alpha=i*0.01
#    predictions=alpha*cnn_predict+(1-alpha)*rnn_predict
#    loss=np.sqrt(mean_squared_error(y_valid,predictions))
    #print('loss:'+str(loss))
#    if(min_loss>loss):
        #print('min_loss:'+str(min_loss))
#        best_alpha=alpha
#        min_loss=loss
#print('best alpha:'+ str(best_alpha))
#print('best loss:'+str(min_loss))


## Predicting the test data
Last and note least it's time to predict the test data 

In [ ]:
%%time
predictions=lm.coef_[0]*model.predict(X_test,batch_size=BATCH_SIZE)+lm.coef_[1]*model2.predict(X_test,batch_size=BATCH_SIZE)+lm.coef_[2]*model3.predict(X_test,batch_size=BATCH_SIZE)+lm.intercept_
predictions=np.exp(predictions)-1
submission=pd.DataFrame(columns=['test_id','price'])
submission['test_id']=test_id
submission['price']=predictions
submission.to_csv('EnsembleSubmission.csv',index=False)

In [ ]:
print('program ended  took '+str(datetime.datetime.now()-time_start)+' minutes')

# Conclusion
So that was my solution I hope that you benefited from it and any feedback about the NN architectures or any questions you wanna ask about the code I would be more than happy to answer it